<a href="https://colab.research.google.com/github/uvsrahul1234/VenkataSaiRahul_INFO5731_Spring2025/blob/main/Unnam_Venkata_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [34]:
# Setup of Selenium and ChromeDriver is offered by this package
!pip install google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 25.6 MB/s eta 0:00:00


In [ ]:
# importing required libraries
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

# Set user-agent headers
chrome_options = Options()

# Setting this header to mimic a real browser
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# loading the driver with header options
driver = gs.Chrome(options=chrome_options)

# url of IMDB website from which review need to be scrapped
url = "https://www.imdb.com/title/tt15239678/reviews/"
driver.get(url) # loading the url using driver

# executing scolling action using selenium
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# adding a delay to make it replicate a human behaviour
time.sleep(2)

reviews_data = []

# clicking all button
all_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button//span[text()='All']"))
    )
driver.execute_script("arguments[0].click();", all_button)
time.sleep(140)  # Short wait for reviews to load

# Locating the review data using XPATH
reviews = WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[@class='ipc-html-content-inner-div']"))
    )

# Appending all reviews to list
for review in reviews:
  reviews_data.append(review.text)

# importing the reviews to dataframe and saving as csv file
df = pd.DataFrame(reviews_data, columns=["review"])
df_subset = df.head(1000) # selecting 1000 reviews
df_subset.to_csv('imdb_reviews.csv', index=False)

print(f"Reviews scraped: {len(reviews_data)} and 1000 reviews Saved to 'imdb_reviews.csv'.")

# closing the drive to release the resourses used
driver.quit()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Reviews scraped: 1007 and 1000 reviews Saved to 'imdb_reviews.csv'.


# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
# Write code for each of the sub parts with proper comments.

# importing libraries
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
from textblob import Word
from nltk.stem import PorterStemmer, WordNetLemmatizer

# loading the reviews csv generated from previous question
df = pd.read_csv('imdb_reviews.csv')

# (a) Remove punctuations.
df['clean_review'] = df['review'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Remove special characters
df['clean_review']  = df['clean_review'].astype(str).apply(lambda x: re.sub(r"[^a-zA-Z0-9\s]", "", x))

# Remove new lines
df['clean_review'] = df['clean_review'].str.replace('\n', ' ', regex=False)

# (b) Remove numbers
df['clean_review']  = df['clean_review'].apply(lambda x: re.sub(r'\d+','', x))

# (C) remove stop words by using the stopwords list
stop = stopwords.words('english')
df['clean_review'] = df['clean_review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# (d) convert text to lower case
df['clean_review'] = df['clean_review'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# (e) Implement Stemming
st = PorterStemmer()
df['clean_review'] = df['clean_review'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# (f) Implement Lemmatization
df['clean_review'] = df['clean_review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

# saving the cleaned data to csv file
df.to_csv('imdb_review_cleaned.csv', index=False)
print("imdb review data has been cleaned and save to new csv file 'imdb_review_cleaned.csv'.")

imdb review data has been cleaned and save to new csv file 'imdb_review_cleaned.csv'.


In [35]:
df.head()

,review,clean_review
0,Had the pleasure to watch this film in an earl...,had pleasur watch film earli screen complet bl...
1,"If you liked or loved the first one, the same ...",if like love first one appli one person i love...
2,"Dune Part 2 is an epic movie; slickly made, an...",dune part epic movi slickli made visual stun b...
3,This is the kind of movie that is impossible t...,thi kind movi imposs justic talk it kind exper...
4,"Like the first part, the second part is visual...",like first part second part visual stun though...


# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [62]:
# Your code here
# Importing libraries
import spacy
import nltk
import pandas as pd
from collections import Counter
from nltk import Tree
from spacy import displacy
from nltk.parse.corenlp import CoreNLPParser

In [63]:
# (a) Parts of Speech (POS) Tagging

# Load the spaCy NLP model

nlp = spacy.load("en_core_web_sm")

# Load CSV file into a DataFrame
csv_file = "imdb_review_cleaned.csv"
df = pd.read_csv(csv_file)

# Drop missing values in the cleaned data column
df['clean_review'].dropna(inplace=True)

# NLP processing to each row
def process_text(text):
    doc = nlp(text)
    pos_counts = Counter()
    for token in doc:
        pos_counts[token.pos_] += 1
    return pos_counts

# saving all POS counts to a column
df['pos_counts'] = df['clean_review'].apply(process_text)

# Expand the pos_counts dictionary into separate columns
pos_columns = ['NOUN', 'VERB', 'ADJ', 'ADV']
for column in pos_columns:
    df[column] = df['pos_counts'].apply(lambda x: x.get(column, 0))

# Drop the pos_counts column as it is no longer needed
df.drop(columns=['pos_counts'], inplace=True)

In [64]:
# getting the sum of total Nouns, Verbs, Adjectives and Adverbs
total_nouns = df['NOUN'].sum()
total_verbs = df['VERB'].sum()
total_adjectives = df['ADJ'].sum()
total_adverbs = df['ADV'].sum()

# print Total POS tags
print(f"Total Nouns: {total_nouns}")
print(f"Total Verbs: {total_verbs}")
print(f"Total Adjectives: {total_adjectives}")
print(f"Total Adverbs: {total_adverbs}")

Total Nouns: 34330
Total Verbs: 16531
Total Adjectives: 13852
Total Adverbs: 4389


In [65]:
# dataframe with Noun, Verb, Adjective and Adverb as seperate columns
df.head()

,review,clean_review,NOUN,VERB,ADJ,ADV
0,Had the pleasure to watch this film in an earl...,had pleasur watch film earli screen complet bl...,131,53,54,20
1,"If you liked or loved the first one, the same ...",if like love first one appli one person i love...,33,20,18,11
2,"Dune Part 2 is an epic movie; slickly made, an...",dune part epic movi slickli made visual stun b...,49,18,22,5
3,This is the kind of movie that is impossible t...,thi kind movi imposs justic talk it kind exper...,18,18,12,7
4,"Like the first part, the second part is visual...",like first part second part visual stun though...,25,7,11,7


In [6]:
!pip install spacy nltk benepar

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.4 MB/s eta 0:00:00
  Created wheel for benepar: filename=benepar-0.2.0-py3-none-any.whl size=37626 sha256=460ae23c2ba317f7f3b530d5c7e2bd389f30e48e1e832280cb4dc791cd87739a
  Stored i

In [47]:
# text sentence that we will do Constituency and Dependency Parsing on..
text = df['clean_review'].dropna().iloc[5]
print(text)

thi second part movi set tone much way first movi make movi feel like cram alot stori backdrop way small time frame which logic consid dune span sever novel histori year civil villeneuv howev put focu aesthet rather storytel which make movi bland confus thing watch the pace irregular part drag alot toward end get feel rap unsatisfi speed especi peopl familiair origin materi dont get sen backstori charact made decis made cram stori like dune two part set film probabl stori justic remad tv seri make realli work therefor rewatch rendit dune unlik


In [53]:
# (2) Constituency and Dependency Parsing
import nltk
from spacy import displacy
import spacy
import benepar
from nltk.tree import Tree

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Load benepar for constituency parsing
benepar.download('benepar_en3')
nlp.add_pipe("benepar", config={"model": "benepar_en3"})

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [56]:
# Process the selected text
doc = nlp(text)

# Dependency Parsing - Extract dependencies
dependency_parse = [(token.text, token.dep_, token.head.text) for token in doc]

# Constituency Parsing - Extract the tree structure
constituency_parse = [sent._.parse_string for sent in doc.sents] if doc.sents else []

# Display Constituency Parsing as a Tree
print("\n🌳 Constituency Parsing Tree:")
for i, tree in enumerate(constituency_parse, 1):
    print(f"Sentence {i}:")
    nltk_tree = Tree.fromstring(tree)  # Convert to nltk Tree format
    nltk_tree.pretty_print()  # Print the tree structure

# Display Dependency Parsing Tree using spaCy displaCy
print("\n🔗 Dependency Parsing Tree (Rendering Below)...")
displacy.render(doc, style="dep", jupyter=True)



🌳 Constituency Parsing Tree:
Sentence 1:
                                                                                                                                                                                                                                                                                         S                                                                                                                                                                                                                                                                                                 
              ___________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________|______________________________                                                                          

In [66]:
# (C) Named Entity Recognition

# Function to extract entities from text and count them
def extract_entities(text):
    doc = nlp(text)
    entity_counts = Counter()

    # Iterate over the entities recognized by spaCy
    for ent in doc.ents:
        entity_counts[ent.label_] += 1  # Increment the count for the entity type

    return entity_counts

# Apply the entity extraction function
df['entity_counts'] = df['clean_review'].apply(extract_entities)

# List of entity types
entity_types = ['PERSON', 'ORG', 'LOC', 'PRODUCT', 'DATE']

# Create new columns for each entity type and add them
for entity_type in entity_types:
    df[entity_type + '_count'] = df['entity_counts'].apply(lambda x: x.get(entity_type, 0))

# dropping the entity_counts column as it is not needed
df.drop(columns=['entity_counts'], inplace=True)

In [67]:
# getting the sum of total Nouns, Verbs, Adjectives and Adverbs
total_persons = df['PERSON_count'].sum()
total_orgs = df['ORG_count'].sum()
total_locs = df['LOC_count'].sum()
total_products = df['PRODUCT_count'].sum()
total_dates = df['DATE_count'].sum()

# printing the total count
print(f"Total Person names: {total_persons}")
print(f"Total Orgs: {total_orgs}")
print(f"Total Locations: {total_locs}")
print(f"Total Product names: {total_products}")
print(f"Total Dates: {total_dates}")

Total Person names: 2437
Total Orgs: 675
Total Locations: 11
Total Product names: 60
Total Dates: 241


In [5]:
df.head()

,review,clean_review,NOUN,VERB,ADJ,ADV,PERSON_count,ORG_count,LOC_count,PRODUCT_count,DATE_count
0,Had the pleasure to watch this film in an earl...,had pleasur watch film earli screen complet bl...,131,53,54,20,9,5,0,0,4
1,"If you liked or loved the first one, the same ...",if like love first one appli one person i love...,33,20,18,11,1,1,0,0,0
2,"Dune Part 2 is an epic movie; slickly made, an...",dune part epic movi slickli made visual stun b...,49,18,22,5,2,1,0,0,0
3,This is the kind of movie that is impossible t...,thi kind movi imposs justic talk it kind exper...,18,18,12,7,1,0,0,0,0
4,"Like the first part, the second part is visual...",like first part second part visual stun though...,25,7,11,7,4,0,0,0,0


# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [22]:
# import necessary libraries
import requests
from bs4 import BeautifulSoup
import csv
import time

# Set the base URL for the GitHub Marketplace for Actions
base_url = "https://github.com/marketplace?page={}&type=actions"

# Set headers to simulate browser request (this helps avoid the 400 error)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

# Function to fetch and parse a page
def fetch_page(page_num):
    url = base_url.format(page_num)
    print(f"Fetching page {page_num}: {url}")
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        print(f"Successfully fetched page {page_num}")
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    else:
        print(f"Failed to fetch page {page_num}. Status code: {response.status_code}")
        return None

# Function to extract product details from a page
def extract_products(soup, page_num):

    products = []

    # Find the marketplace items
    items = soup.find_all('div', class_='position-relative border rounded-2 d-flex marketplace-common-module__marketplace-item--MohVH gap-3 p-3')  # Update the selector based on actual structure
    print(f"Found {len(items)} items on page {page_num}")

    for item in items:

        link_tag = item.find('a', class_='marketplace-common-module__marketplace-item-link--jrIHf line-clamp-1')
        # Extract data from each item
        link_tag1 = item.find('p', class_='mt-1 mb-0 text-small fgColor-muted line-clamp-2')

        # getting product name, description and url
        if link_tag and link_tag1:
          product_name = link_tag.get_text(strip=True)
          description =  link_tag1.get_text(strip=True)
          url = "https://github.com" + link_tag['href']

          print(f"Extracted product: {product_name}, {description}, {url}")  # Print extracted product details

        # Construct the product information and append it to the products list
        products.append({
            "product_name": product_name,
            "description": description,
            "url": url,
            "page_number": page_num
        })

    return products

# Function to write the data into a CSV file
def write_to_csv(products, filename='github_marketplace_actions.csv'):
    header = ['product_name', 'description', 'url', 'page_number']

    # Write data to CSV file
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=header)

        # Write header only once
        if file.tell() == 0:
            writer.writeheader()

        writer.writerows(products)

# Main function to scrape GitHub Marketplace Actions
def scrape_github_marketplace_actions(pages=53, delay=3):
    for page_num in range(1, pages + 1):
        # Fetch the page
        soup = fetch_page(page_num)

        if soup:
            # Extract product details
            products = extract_products(soup, page_num)

            # Check if there are any products before writing to CSV
            if products:
                write_to_csv(products)
            else:
                print(f"No products found on page {page_num}.")

            # Delay to prevent overloading the server
            print(f"Waiting for {delay} seconds before fetching the next page...")
            time.sleep(delay)

if __name__ == "__main__":
    # Start scraping 50 pages as each page contains 20 products
    scrape_github_marketplace_actions(pages=53, delay=3)


Fetching page 1: https://github.com/marketplace?page=1&type=actions
Successfully fetched page 1
Found 20 items on page 1
Extracted product: TruffleHog OSS, Scan Github Actions with TruffleHog, https://github.com/marketplace/actions/trufflehog-oss
Extracted product: Metrics embed, An infographics generator with 40+ plugins and 300+ options to display stats about your GitHub account, https://github.com/marketplace/actions/metrics-embed
Extracted product: yq - portable yaml processor, create, read, update, delete, merge, validate and do more with yaml, https://github.com/marketplace/actions/yq-portable-yaml-processor
Extracted product: Super-Linter, Super-linter is a ready-to-run collection of linters and code analyzers, to help validate your source code, https://github.com/marketplace/actions/super-linter
Extracted product: Gosec Security Checker, Runs the gosec security checker, https://github.com/marketplace/actions/gosec-security-checker
Extracted product: Rebuild Armbian and Kernel, 

In [30]:
# import necessary libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import warnings

# ignore filter warnings
warnings.filterwarnings("ignore")

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

df = pd.read_csv('github_marketplace_actions.csv')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function for text preprocessing
def clean_text(text):

    if pd.isna(text):  # Check if text is NaN and replace it with an empty string
        return ""

    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a string
    return " ".join(tokens)

# Apply cleaning function to the DataFrame column
df['cleaned_description'] = df['description'].apply(clean_text)

print("github_marketplace_actions data has been cleaned")

github_marketplace_actions data has been cleaned


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [31]:
df.head()

,product_name,description,url,page_number,cleaned_description
0,TruffleHog OSS,Scan Github Actions with TruffleHog,https://github.com/marketplace/actions/truffle...,1,scan github action trufflehog
1,Metrics embed,An infographics generator with 40+ plugins and...,https://github.com/marketplace/actions/metrics...,1,infographics generator plugins option display ...
2,yq - portable yaml processor,"create, read, update, delete, merge, validate ...",https://github.com/marketplace/actions/yq-port...,1,create read update delete merge validate yaml
3,Super-Linter,Super-linter is a ready-to-run collection of l...,https://github.com/marketplace/actions/super-l...,1,superlinter readytorun collection linters code...
4,Gosec Security Checker,Runs the gosec security checker,https://github.com/marketplace/actions/gosec-s...,1,run gosec security checker


In [32]:
# import necessary libraries
import pandas as pd

# 1. Check for Missing Values
missing_values_productName = df['product_name'].isnull().sum()
missing_values_description = df['description'].isnull().sum()
missing_values_url = df['url'].isnull().sum()
missing_values_cleaned_description = df['cleaned_description'].isnull().sum()
print("Missing Values per Column:\n", missing_values_productName)
print("Missing Values per Column:\n", missing_values_description)
print("Missing Values per Column:\n", missing_values_url)
print("Missing Values per Column:\n", missing_values_cleaned_description)

# removing missing values
df.dropna(inplace=True, axis=0)
print("Removed the rows if we have missing values")

# 2. Remove Duplicate Rows
df_duplicates = df.duplicated().sum()
df = df.drop_duplicates()
print(f"Number of duplicate rows removed: {df_duplicates}")

# Save the cleaned and validated dataset
df.to_csv('cleaned_github_marketplace_actions.csv', index=False)

print("\n✅ Data quality checks completed successfully!")

Missing Values per Column:
 0
Missing Values per Column:
 1
Missing Values per Column:
 0
Missing Values per Column:
 0
Removed the rows if we have missing values
Number of duplicate rows removed: 0

✅ Data quality checks completed successfully!


#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [4]:
# installing tweepy
!pip install tweepy

In [7]:
# import necessary packages
import tweepy
import pandas as pd

# using google colab secret to store the token
from google.colab import userdata
bearer_token = userdata.get('TWITTER_TOKEN1')

# setting the limit on tweets and setting up client with bearer token
client = tweepy.Client(bearer_token=bearer_token)
no_of_tweets = 100

# query used to fetch the data, here we exclude retweets and replies and the language is english
query = "#artificialintelligence -is:retweet -is:reply lang:en"
tweets = client.search_recent_tweets(query=query, tweet_fields=["text", "author_id"], user_fields=["username"] , expansions = ["author_id"], max_results=no_of_tweets)

In [8]:
# create a userdata variable to store respective usernames
users_data = {}
if tweets.includes and 'users' in tweets.includes:
  users_data.update({u.id: u.username for u in tweets.includes['users']})

# print to check the output data
for tweet in tweets.data:
    username = users_data.get(tweet.author_id, "Unknown User")
    print(f"User: {username} \nAuthor_id:{tweet.author_id}\nTweet: {tweet.text},\nTweet ID: {tweet.id} \n")

User: SteveKlinko 
Author_id:726415079516852225
Tweet: #Consciousness Chips will upgrade Robots to Androids. https://t.co/BXfRQY5coW #ArtificialIntelligence #AI #Philosophy #Engineering,
Tweet ID: 1892391558424428561 

User: Every1Just1970 
Author_id:1875497979886628865
Tweet: Grok vs ChatGPT vs DeepSeek: Battle of AI Giants #ArtificialIntelligence #ChatGPT #Grok #DeepSeek #TechComparison #AITechnology #FutureOfAI #TechNews #ElonMusk #OpenAI #DigitalTransformation #AIAssistant #TechTrends #Innovation

https://t.co/g2rfLBaWG5,
Tweet ID: 1892390064971227254 

User: techjunkiejh 
Author_id:3064266184
Tweet: #ElonMusk’s #xAI releases its latest flagship model, #Grok3 | TechCrunch https://t.co/wj4hxWIUG8  #TechJunkieInvest #investing #TechJunkieNews #Tech #ArtificialIntelligence #AIModel https://t.co/0wjGnX6aFP,
Tweet ID: 1892389725803003919 

User: BDONonprofit 
Author_id:482192892
Tweet: While embracing advancements in AI technology, it’s crucial for higher ed professionals to prioritize 

In [10]:
# Create a list to store tweet data
tweet_data = []

# loop to add tweets data to the list
for tweet in tweets.data:
  tweet_data.append({
    'tweet_id': tweet.id,
    'username': users_data.get(tweet.author_id, "Unknown User"),
    'text': tweet.text
    })

# Convert the list of tweet data to a pandas DataFrame
df = pd.DataFrame(tweet_data)

# Display the DataFrame
print(df)

# Save DataFrame to a CSV file
df.to_csv("tweets_AI.csv", index=False)

               tweet_id        username  \
0   1892391558424428561     SteveKlinko   
1   1892390064971227254  Every1Just1970   
2   1892389725803003919    techjunkiejh   
3   1892389229948289251    BDONonprofit   
4   1892388959944118466    randydeutsch   
..                  ...             ...   
95  1892350106470322194      Luan_hack1   
96  1892350017207210289      Luan_hack1   
97  1892349949385306353      Luan_hack1   
98  1892349907744305338      Luan_hack1   
99  1892349757139386797      Luan_hack1   

                                                 text  
0   #Consciousness Chips will upgrade Robots to An...  
1   Grok vs ChatGPT vs DeepSeek: Battle of AI Gian...  
2   #ElonMusk’s #xAI releases its latest flagship ...  
3   While embracing advancements in AI technology,...  
4   Chaos acquires CEO Bill Allen's AI software fi...  
..                                                ...  
95  For any lost coins, wallet, pass phrase or Any...  
96  I am trusted and Reliable for a

In [11]:
# importing necessary libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

# loading tweets saving in csv
df = pd.read_csv("tweets_AI.csv")

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function for text preprocessing
def clean_text(text):
    if pd.isna(text):  # Handle NaN values
        return ""

    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)

    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a string
    return " ".join(tokens)

# Apply cleaning function to the 'text' column
df['cleaned_text'] = df['text'].apply(clean_text)

# Display cleaned dataset
df[['text', 'cleaned_text']]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,text,cleaned_text
0,#Consciousness Chips will upgrade Robots to An...,consciousness chip upgrade robot android artif...
1,Grok vs ChatGPT vs DeepSeek: Battle of AI Gian...,grok v chatgpt v deepseek battle ai giant arti...
2,#ElonMusk’s #xAI releases its latest flagship ...,elonmusks xai release latest flagship model gr...
3,"While embracing advancements in AI technology,...",embracing advancement ai technology crucial hi...
4,Chaos acquires CEO Bill Allen's AI software fi...,chaos acquires ceo bill allen ai software firm...
...,...,...
95,"For any lost coins, wallet, pass phrase or Any...",lost coin wallet pas phrase account problem co...
96,I am trusted and Reliable for any Hacking or A...,trusted reliable hacking account recovery arti...
97,Hacking is easy only with the professional hac...,hacking easy professional hacker contact suppo...
98,Hacking is easy only with the professional hac...,hacking easy professional hacker contact suppo...


In [12]:
# data quality operations

print(f"Number of duplicate rows: {df.duplicated().sum()}")
df = df.drop_duplicates()  # Remove duplicates if found

# checking datatype integrity
print("\nData Types of Each Column:")
print(df.dtypes)

# print missing columns and remove if any
print("Missing Values per Column:\n", df.isnull().sum())

# checking the numberm of unique tweet ID's
if 'tweet_id' in df.columns:
    unique_ids = df['tweet_id'].nunique()
    total_ids = df.shape[0]
    print(f"\nUnique Tweet IDs: {unique_ids} / Total Rows: {total_ids}")
    if unique_ids < total_ids:
        print("Warning: Duplicate Tweet IDs detected!")

# saving cleaned tweets to csv
df.to_csv('tweets_AI_cleaned.csv', index=False)
print("\n✅ Cleaned data saved successfully!")


Number of duplicate rows: 0

Data Types of Each Column:
tweet_id         int64
username        object
text            object
cleaned_text    object
dtype: object
Missing Values per Column:
 tweet_id        0
username        0
text            0
cleaned_text    0
dtype: int64

Unique Tweet IDs: 100 / Total Rows: 100

✅ Cleaned data saved successfully!


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog

#### I really liked doing webscrapping using selenium and beautiful soap. By working on different questions I was able to learn a lot on both the methods. The questions this time are really great and there was sufficient time to solve them. In next assignments maybe working on LLM's would be really great. I worked on preprocessing of numerical data, but this is the first time working on text data. Finding POS and NER tags in sentences was exciting. Dependency and constituency parsing was a bit hard to understand. I wonder whether the tagging is correct or not? Anyways, by doing first 3 questions without using AI gave a lot of Knowledge. By doing the twitter Question now I am confident of using API to fetch data, as i had to go through a lot of documentation to find the right parameters. Overall This is ONE OF THE BEST ASSIGNMENT I have done in a while.